# Access HRRR Grib2 data via ReferenceFileSystem

In [ ]:
import fsspec
import xarray as xr
import cfgrib
import hvplot.xarray

#### Read temperature file with cfgrib

In [ ]:
url = 's3://noaa-hrrr-bdp-pds/hrrr.20190101/conus/hrrr.t14z.wrfsfcf01.grib2'
file = fsspec.open(url)

tmp_file = fsspec.open_local(f'simplecache::{url}', 
                              s3=dict(anon=True), simplecache={'cache_storage': './tmp'})

In [ ]:
%%time

ds = xr.open_dataset(tmp_file, engine='cfgrib', 
                     backend_kwargs=dict(filter_by_keys={'typeOfLevel':'heightAboveGround', 'level':2}))

In [ ]:
ds.t2m

In [ ]:
%%time
ds.t2m.mean().values

#### Read with ReferenceFileSystem

Add grib decoder

In [ ]:
fs = fsspec.filesystem('https')
fs.download('https://gist.githubusercontent.com/rsignell-usgs/31a3de1754726990803e8871ef2e1598/raw/2aa7f056a0fc1a38bd2c8c02f20f340394eebc68/grib2.py', 'grib2.py')
%run grib2.py

In [ ]:
%%time
so = {"anon": True}
fo = "hrrr_total.json"
fs = fsspec.filesystem("reference", fo=fo, target_options=so,
                       remote_protocol="s3", remote_options=so)
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr")

In [ ]:
ds.t2m

In [ ]:
ds.t2m[:,400,400].hvplot(x='time', grid=True)

In [ ]:
%%time
ds.t2m.mean().values

In [ ]:
print(fs)

In [ ]:
ds.t2m.nbytes/1e6

In [ ]:
ds = ds.assign_coords(longitude=(((ds.longitude + 180) % 360) - 180))

In [ ]:
ds.t2m.hvplot.quadmesh(x='longitude', y='latitude', 
                       rasterize=True, geo=True, cmap='turbo', tiles='OSM')